# Ajout de l'animation

Deja, on ajoute le déplacement vers la première intersection :

In [ ]:
# On renomme 'continue_to_travel' en 'progress_into_the_path' qui est plus logique
def progress_into_the_path(self, backward_direction=NOWHERE): # NOWHERE c'est quand on veut parcourir en partant de rien
    # ...

# Au moment ou on creer le perso, on le fait traverser
perso = Perso(niveau)
perso.progress_into_the_path()

Ensuite la on va gérer l'animation, du coup là on doit faire en sorte que la boucle principale avec l'affichage se face en parallele qu'on fait la fonction travel(), ca devient le bordel.

La manière la plus logique de faire ca c'est avec les threads, on crée 2 processus en parallele, un qui affiche et l'autre qui déplace le perso (fait du traitement). 
Le problème c'est que ca va complexifié l'appli et si la structure est pas propre (ce qui est le cas ici :P), ca sera pas maintenable si on veut rajouter des trucs.
Par exemple, il faut (en plus des problème de concurrence) ralentir le déplacement du perso avec le clockrate, ou encore faire du controle avec des booleens (genre un qui dit qu'on se déplace, donc on doit bloquer les evenement etc ...)

Je vais faire une autre approche, un peu plus subtile qui n'utilisera pas les threads. La voici :

Je rajoute un objet JobManager, qui va contenir les taches (les jobs) à effectuer à chaque tour de boucle.
La fonction travel va simplement rajouter autant de tache qu'il y a de déplacement dans le parcours.

Ca a l'avantage de quasiment pas changer la logique du code.

Ca fait un truc du style :

In [ ]:
class JobManager:
    def __init__(self):
        self.jobs = deque() # Une file

    def add_job(self, job):
        self.jobs.append(job)

    def execute_next_job(self):
        job = self.jobs.popleft() # recup le prochain
        job() # execute le job

    def has_job(self):
        return len(self.jobs) # 0 == False


Du coup, dans la boucle du jeu, si on a une tache à faire, on l'execute, sinon on fait les evenements comme avant

In [ ]:
while continuer_jeu:

    if job_manager.has_job():            # Si on a un job
        job_manager.execute_next_job()   # on l'execute
    
    # On traite les evenements    
    for event in pygame.event.get(): 
        # ...
    
    # On gere l'affichage
    fenetre.blit(fond, (0, 0))
    # ...


Maintenant, reste plus qu'à ajouter ces jobs dans le travel()

In [ ]:
def travel(self, direction):
    self.move(direction):
            
    backward_direction = opposite_direction(direction)
    
    job = lambda: self.progress_into_the_path(backward_direction)
    self.job_manager.add_job(job)


Ici, la seule différence à avant, c'est qu'on n'execute pas directement progress_into_the_path(), mais on met la methode dans une fonction lambda (fonction anonyme), qui sera executer plus tard, dans le prochain tour de boucle.

Du coup, c'est dans le :

    job_manager.execute_next_job()
    
que progress_into_the_path() sera executé. 

Comme cette fonction va relancer travel, cela va la rajouter encore une fois dans les jobs, et ainsi de suite, jusqu'a ce qu'il n'y a plus de déplacement possible.

In [ ]:
def progress_into_the_path(self, backward_direction=NOWHERE):
    # ...
    
    # S'il n'y a qu'une seule direction possible
    if len(possible_direction) == 1:
        #...
        
        # on continue le parcours
        self.travel(direction) # va remettre le parcours dans les jobs

Après, la rapidité de l'animation est entièrement basé sur le clockrate :

    pygame.time.Clock().tick(30)